## Titanic - Analysis and Predict 
This is my first Notebook in Kaggle, then if you find any error tell me, please.

In [ ]:
# Manipulation of vectors library
import numpy as np
# Manipulation of data library
import pandas as pd
#Ploting graphs library
import matplotlib.pyplot as plt
import seaborn as sns
# Manipulation of the system library
import os

Let's see the files on kaggle/input/titanic folder.

In [ ]:
! cd /kaggle/input/titanic && ls

Now, define the train and test variables from system folder.

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

Some data of train DataFrame.

In [ ]:
df_train.head()

## Describing Data

Let's begin with some statistics.

In [ ]:
df_train.describe().transpose()

Realize that there's some missing values on the data, we have to handle with them. Also, 38.38% of passengers survived, so let's compare the survival rate with the others columns.

In [ ]:
df_train.groupby('Pclass').mean()['Survived']*100

Notice that, the survival rate grow with de class of the passager. 

Moreover, the data have some missing data values e we'll handle with them.

In [ ]:
df_train.isna().sum()

The correlation is

In [ ]:
df_train.corr()

But, this is hard to vizualize, then, let's see this using an heatmap plot from seaborn.

## Ploting Data

Heatmap of correlation

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df_train.corr(), annot=True, cmap='viridis')
plt.show()

With this correlation map maybe:


* Fare is inverse of Pclass, then passagers of low class paid less for the trip;
* SibSp is proportional to Parch, this is expected;
* Age is inversely proportional to class, maybe the upper class is more elderly than the lower, we'll se it below;
* Pclass is inverse of Survived, this is alredy expected once the class 1 have 62.96% rate of survived;
* Fare is proportional to Survived, maybe who paid more for the trip belong to class 1, thus have more chances to survive;

  Let's response these questions.

* Fare is inverse of Pclass, then passagers of low class paid less for the trip;


YES, as you can see, classes 2 and 3 paid less for the trip. Moreover, for the class 1 there's some peoples that paid over than 200.

In [ ]:
print(df_train.groupby('Pclass').mean()['Fare'])
plt.figure(figsize=(10,8))
sns.boxplot(x='Pclass', y='Fare', data=df_train)
plt.title('Pclass x Fare')
plt.show()

* SibSp is proportional to Parch, this is expected;


In [ ]:
plt.figure(figsize=(16,9))
sns.barplot(x='SibSp', y='Parch', data=df_train)
plt.title('SubSp x Parch')
plt.show()

* Age is inversely proportional to class, maybe the upper class is more elderly than the lower, we'll se it below;


Affirmative. As you can see, the age is inversely proportinal to class.

In [ ]:
print(df_train.groupby('Pclass').mean()['Age'])
plt.figure(figsize=(16,9))
sns.barplot(x='Pclass', y='Age', data=df_train)
plt.title('Pclass x Age')
plt.show()

Pclass is inverse of Survived, this is alredy expected once the class 1 have 62.96% rate of survived;

As alredy expected.

In [ ]:
print(df_train.groupby('Pclass').mean()['Survived'])
plt.figure(figsize=(16,9))
sns.barplot(x='Pclass', y='Survived', data=df_train)
plt.title('Pclass x Survived')
plt.show()

* Fare is proportional to Survived, maybe who paid more for the trip belong to class 1, thus have more chances to suvive;

Again, yes. But, realize that the columns, Fare and Pclass have strong relationship with Survived. Thus, Who belongs to the upper class or paid more for the trip probably survived.

In [ ]:
print(df_train.groupby(['Pclass', 'Survived']).mean()['Fare'])
sns.FacetGrid(df_train, col='Pclass', height=5, aspect=4/3).map(sns.barplot, 'Survived', 'Fare', order=[0, 1])
plt.show()

They are very correlated, almost linear, that means that the feature importance is directly related to the outcome. Certainly Pclass and Fare have lot importance for predict who survived. Therefore, our model need to learn this.

In [ ]:
plt.figure(figsize=(16*(2/3),9*(2/3)))
sns.heatmap(df_train[['Fare', 'Survived', 'Pclass']].corr(), cmap='viridis', annot=True)
plt.title('Correlation: Fare x Survived x Pclass')
plt.show()

Also, most of survived is woman (74.20%).

In [ ]:
print(df_train.groupby('Sex').mean()[['Survived']]*100)
plt.figure(figsize=(16,9))
sns.countplot(x='Sex', hue='Survived', data=df_train, palette='viridis')
plt.title('Count of Survive by Sex')
plt.show()

## Treating Missing Values

We have to handle with these missing values, let's see this again.

In [ ]:
print(df_train.isna().sum(),'\n\n',df_test.isna().sum())

Begin treating the column Embarked.

Let's try to fit the NaN into a Embarked.

In [ ]:
df_train[df_train['Embarked'].isna() == True]

Using a barplot to vizualize some columns. We can't label the missing data on 'S', 'C' or 'Q'. Let's try other way to do this.

In [ ]:
plt.figure(figsize=(16,9))
sns.FacetGrid(df_train, col='Embarked', row='Sex', height=4, aspect=.8).map(sns.barplot, 'Pclass', 'Fare', order=[1, 2, 3])
plt.show()

As you can see below, most of people who embark in the "Q" belong to class 3 and have paid 13.28 $\pm$ 14.18, may the missing data not belong to "Q". But is not possible to go so far, then the missing data values will fit in Embarked "S", because this is more commom than Embarked "C".

In [ ]:
df_train.groupby('Embarked').agg(['mean', 'std']).drop('PassengerId', axis=1)

In [ ]:
#For train data
df_train['Embarked'] = df_train['Embarked'].fillna(value='S')

#For test data
df_test['Embarked'] = df_test['Embarked'].fillna(value='S')

Then, apply get_dummies to separete the Embarked colum.

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], drop_first=True)
df_test = pd.get_dummies(df_test, columns=['Embarked'], drop_first=True)

Now, let's handle with Cabin column. Couting the NaN data.

In [ ]:
df_train.Cabin.isna().sum()

There's some repeated values.

In [ ]:
df_train.Cabin.value_counts()

As you can see 77.10% of da Cabin column is missing data and apparently there's no relationship between this column and the Survived, so let's just drop it.

In [ ]:
df_train.Cabin.isna().sum() / len(df_train.Cabin) * 100

Then, we drop this for train and test data.

In [ ]:
#For train and test data
df_train = df_train.drop('Cabin', axis=1)
df_test = df_test.drop('Cabin', axis=1)

Now, we treating the colum Sex using get_dummies.

In [ ]:
#for train data
df_train = pd.get_dummies(df_train, columns=['Sex'], drop_first=True)

#for test data
df_test = pd.get_dummies(df_test, columns=['Sex'], drop_first=True)

At leats, let's handle with Age missing data.

Highest class is more elders.

In [ ]:
plt.figure(figsize=(16,9))
sns.boxplot(x='Pclass', y='Age', data=df_train)
plt.title('Pclass x Age')
plt.show()

To solve missing data problem, let's build up a regression model using Keras from Tensorflow.

But before drop columns of string type.

In [ ]:
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

In [ ]:
df_train.hist()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Use Normalizer to normalize data, this bring all variable to same range [0,1].

In [ ]:
norm = MinMaxScaler()

We use the train DataFrame with not missing data to do regression, then we apply it on de missing values of train and test DataFrame.

In [ ]:
df_train_notna = df_train[df_train['Age'].notna() == True].drop(['Survived'], axis=1)

df_X = df_train_notna.drop(['Age'], axis=1)
df_y = df_train_notna['Age']

Split the data into train data and test data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=72)

Apply normalization into X_train and X_test.

In [ ]:
norm.fit(X_train)
X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

Import TensorFlow, for training a deep neural networks to predict the age.

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=[7]),
    #This sequential begin with 10 neurons, the activation is ReLu and the input shape is the number of columns of our training dataset.
    
    tf.keras.layers.BatchNormalization(),
    #Batch normalization applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1.
    
    tf.keras.layers.Dense(15, activation='relu'),
    #Add 15 neurons using ReLu as activation function
    
    tf.keras.layers.Dense(1)
    #The output must be 1 label for a regression model
])

A summary of our model.

In [ ]:
model.summary()

* SGD, stochastic gradient descent, optimizer is a method used to minimize the cost fuction;
* Mean squared logarithmic error, MSLE is a function that measure the ration between the true and predict values;
* The metrics accuracy and MSE(Mean Squared Error) measure the behavior of test and validation data along the fitting;
* EarlyStopping is a callback used to avoid overfitting of the data;

In [ ]:
model.compile(optimizer='SGD',loss='mean_squared_logarithmic_error',metrics=['accuracy', 'MSE'])
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_train, y_train), callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss'))

In [ ]:
plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['loss'], 'r', label='loss')
plt.plot(history.history['val_loss'], 'b', label='val_loss')
plt.title('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['accuracy'], 'r', label='accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='val_accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['MSE'], 'r', label='MSE')
plt.plot(history.history['val_MSE'], 'b', label='val_MSE')
plt.title('MSE')
plt.legend()
plt.show()

This is a little bad, but is the best that we have. There's three ways to continue, use this regreesion for age on the missing data, use mean age to fill them or delete them. At the end of the notebook we'll try these three ways to handle with this, but now with notebook using regression.

In [ ]:
classifications = model.predict(X_test)
print(f'Correlation Matrix:\n{np.corrcoef(classifications[:,0], y_test)}')
plt.figure(figsize=(16,9))
sns.scatterplot(x = classifications[:,0], y = y_test)
sns.lineplot(x = y_test, y = y_test, color='b')
plt.xlim(.94*min(classifications[:,0]), 1.02*max(classifications[:,0]))
plt.xlabel('Predicted Age')
plt.ylabel('Real Age')
plt.show()

Then, we apply our model into our train and test data.

In [ ]:
df_train['Age'] = df_train['Age'].replace({np.nan: np.array(model.predict(norm.fit_transform(df_train.drop(['Age', 'Survived'], axis=1))))})
df_test['Age'] = df_test['Age'].replace({np.nan: np.array(model.predict(norm.fit_transform(df_test.drop(['Age'], axis=1))))})

In [ ]:
print(df_train.isna().sum(),'\n\n',df_test.isna().sum())

Let's just use mean values on the Fare missing data.

In [ ]:
df_test['Fare'] = df_test['Fare'].fillna((df_test.Fare.mean()))
df_train['Fare'] = df_train['Fare'].fillna((df_train.Fare.mean()))

## Modelling Deep Neural Network for Predict Survival

We begin sorting out train and test data.

In [ ]:
X_train = df_train.drop('Survived',axis=1)
y_train = df_train['Survived']
X_test = df_test

Normalizing data.

In [ ]:
norm = MinMaxScaler()
norm.fit(X_train)
X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

Let's build model using keras.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=[8]),
    #This sequential begin with 10 neurons, the activation is ReLu and the input shape is the number of columns of our training dataset.
    
    tf.keras.layers.BatchNormalization(),
    #Batch normalization applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1.
    
    tf.keras.layers.Dense(14, activation='relu'),
    #Add 14 neurons using ReLu as activation function
    
    tf.keras.layers.Dense(1, activation='sigmoid')
    #The output must be 1 label and de activation is a sigmoid function for binary classification
])

In [ ]:
model.summary()

* Binary crossentropy is a loss function/
* SGD, stochastic gradient descent, optimizer is a method used to minimize the cost fuction;
* The metrics accuracy and MSE(Mean Squared Error) measure the behavior of test and validation data along the fitting;
* EarlyStopping is a callback used to avoid overfitting of the data monitoring the value of loss;

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),metrics=['accuracy', 'MSE'])
history = model.fit(X_train, y_train, epochs=500, validation_split=.2, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss'))

In [ ]:
plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['loss'], 'r', label='loss')
plt.plot(history.history['val_loss'], 'b', label='val_loss')
plt.title('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['accuracy'], 'r', label='accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='val_accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(16*.7,9*.7))
plt.plot(history.history['MSE'], 'r', label='MSE')
plt.plot(history.history['val_MSE'], 'b', label='val_MSE')
plt.title('MSE')
plt.legend()
plt.show()

DNN is not the best way to handle with this type of problem. Also, there is not much data to build up a good model. However, let's build up the same model above using all training data, i.e, validation_split=0.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),metrics=['accuracy', 'MSE'])
history = model.fit(X_train, y_train, epochs=500, validation_split=0, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss'))

## Prediction & Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
classifications = model.predict(X_train)
for i, ival in enumerate(classifications):
    if ival >= 0.5: classifications[i] = 1
    else: classifications[i] = 0
print(f'Correlation Matrix:\n{np.corrcoef(classifications[:,0], y_train)}\n\n')
print(f'Confusion Matrix:\n{confusion_matrix(y_train, classifications)}\n\n')
print(f'Accuracy Matrix:\n{accuracy_score(y_train, classifications)}')

In [ ]:
df_test_X = norm.fit_transform(df_test)
df_test_y = model.predict(df_test_X)
for i, ival in enumerate(df_test_y):
    if ival >= 0.5: df_test_y[i] = 1
    else: df_test_y[i] = 0

In [ ]:
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv", index_col='PassengerId')
submission['Survived'] = df_test_y.astype(int)
submission.to_csv('TitanicKNN.csv', index=False)

## Baselines

In [ ]:
X_test.shape, y_test.shape

In [ ]:
df_train

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

Machine Learning Algorithm

In [ ]:
MLA = [
    #SVM
    ('SVC', SVC()),
    ('LinearSVC', LinearSVC()),
    
    #Tree
    ('DecisionTreeClassifier', DecisionTreeClassifier()),
    ('ExtraTreeClassifier', ExtraTreeClassifier()),
    
    #Linear Model
    ('LogisticRegression', LogisticRegressionCV()),
    
    #Neighbors
    ('KNeighborsClassifier', KNeighborsClassifier()),
    
    #Gaussian Process
    ('GaussianProcessClassifier', GaussianProcessClassifier()),
    
    #Ensemble
    ('AdaBoostClassifier', AdaBoostClassifier()),
    
    #Naive Bayes
    ('GaussianNB', GaussianNB())
]

In [ ]:
X = np.array(df_train.drop('Survived', axis=1))
y = np.array(df_train['Survived'])

In [ ]:
shape = (10,10)
results = np.zeros((len(MLA), shape[0], shape[1]))
for count, algorithm in enumerate(MLA):
    cv = RepeatedKFold(n_splits=10, n_repeats=10)
    model = Pipeline([('scaler', MinMaxScaler()), algorithm])
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=1)
    results[count] = scores.reshape(shape)

The plot below represent a boxplot of each ML Algorithm with 10 repeats using KFold. The horizontal lines represents the mean of scores and the (mean +/- std) of scores.

In [ ]:
for index, result in enumerate(results):
    plt.title(MLA[index][0])
    plt.boxplot(result, showmeans=True)
    plt.axhline(np.mean(result), c='r')
    plt.axhline(np.mean(result) + np.std(result), c='r', ls='--')
    plt.axhline(np.mean(result) - np.std(result), c='r', ls='--')
    plt.show()

Looking at best result of Machine Learning Algorithms

In [ ]:
best_mean = [np.mean(x) for x in results]
index_best_mean = best_mean.index(max(best_mean))
print(f'mean of best result ({MLA[index_best_mean][0]})')
for i in results[index_best_mean]:
    print(f'{np.round(np.mean(i), decimals=3)} +/- {np.round(np.std(i), decimals=3)}')
